In [4]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import time

driver = webdriver.Chrome()

driver.get('https://www.redbus.in/');

time.sleep(5)

1. Telangana State


In [1]:
#Import Statements
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd

# Initialize the WebDriver
def initialize_driver():
    driver = webdriver.Chrome() # Opens the Chrome browser.
    driver.maximize_window() # Maximizes the browser window.
    return driver # Returns the driver instance.


# Load a webpage
def load_page(driver, url):
    driver.get(url) # Opens the specified URL in the browser.
    time.sleep(5)  # Waits for 5 seconds to ensure the page loads fully.


# Scrape all bus routes on the current page (or) Scrape Bus Routes
def scrape_bus_routes(driver):
    routes = driver.find_elements(By.CLASS_NAME, 'route')  # Finds all elements with the class `route`.
    route_links = [route.get_attribute('href') for route in routes] # Extracts href links from the elements.
    route_names = [route.text.strip() for route in routes] # Extracts the visible text (route names).
    return zip(route_links, route_names) # Returns a zipped object of links and names.


# Scrape bus details for a specific route
def scrape_bus_details(driver, url, route_name):
    driver.get(url) # Navigates to the route's URL.
    time.sleep(5) # Waits for the page to load.
    
    try:
        # Click "View Buses" button if available
        view_buses = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.CLASS_NAME, "button"))
        )
        view_buses.click()
        time.sleep(5) # Waits for buses to load.
        
        # Scroll to load buses  # Scrolls to the bottom of the page to load additional content.
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(5)
    except:
        print(f"No 'View Buses' button found for {route_name}")
        return [] # Returns an empty list if the button isn't found.
        

    # Collect bus details
    buses = driver.find_elements(By.CLASS_NAME, 'bus-item')  # Finds all bus items on the page.
    bus_details = []
    for bus in buses:  # Iterates through each bus element.
        try:
            bus_details.append({
                "Route_Name": route_name,
                "Route_Link": url,
                "Bus_Name": bus.find_element(By.CLASS_NAME, 'travels').text,  # Extracts bus name.
                "Bus_Type": bus.find_element(By.CLASS_NAME, 'bus-type').text,  # Extracts bus type.
                "Departing_Time": bus.find_element(By.CLASS_NAME, 'dp-time').text,  # Extracts departure time.
                "Duration": bus.find_element(By.CLASS_NAME, 'dur').text,  # Extracts travel duration.
                "Reaching_Time": bus.find_element(By.CLASS_NAME, 'bp-time').text, # Extracts arrival time.
                "Star_Rating": bus.find_element(By.CLASS_NAME, 'rating-sec').text if bus.find_elements(By.CLASS_NAME, 'rating-sec') else '0', # Extracts rating or sets it to '0'.
                "Price": bus.find_element(By.CLASS_NAME, 'fare').text,  # Extracts ticket price.
                "Seat_Availability": bus.find_element(By.XPATH, ".//div[contains(@class, 'seat-left')]").text if bus.find_elements(By.XPATH, ".//div[contains(@class, 'seat-left')]") else '0'
                 # Extracts seat availability or sets it to '0'.
            })
        except Exception as e:
            print(f"Error extracting bus details: {e}")
    return bus_details # Returns a list of dictionaries containing bus details.


# Scrape bus details from all pages
def scrape_all_pages(url, pages=3):
    driver = initialize_driver()  # Initializes the browser.
    all_details = [] # List to store all scraped bus details.

    try:
        for page in range(1, pages + 1): # Loops through pages (default: 3 pages).
            load_page(driver, url) # Loads the main page.
            if page > 1: # If it's not the first page:
                try:
                    # Find and click the pagination tab for the current page.
                    pagination = driver.find_element(By.XPATH, f"//div[contains(@class, 'DC_117_pageTabs') and text()='{page}']")
                    pagination.click()
                    time.sleep(5) # Waits for the next page to load.
                except Exception as e:
                    print(f"Error navigating to page {page}: {e}")
                    continue # Skips to the next iteration if there's an error.
            
            # Scrape routes and details for the current page
            for route_link, route_name in scrape_bus_routes(driver):
                all_details.extend(scrape_bus_details(driver, route_link, route_name))
    finally:
        driver.quit() # Closes the browser at the end of the scraping process.
    return all_details # Returns the complete list of scraped details.


# Main execution
if __name__ == "__main__":
    URL = "https://www.redbus.in/online-booking/tsrtc/?utm_source=rtchometile"
    bus_details = scrape_all_pages(URL) # Scrapes all pages for bus details.

    # Save the scraped data to a CSV file
    if bus_details:
        pd.DataFrame(bus_details).to_csv('Telangana_bus_details.csv', index=False) # Saves data to CSV.
        print("Scraping completed and data saved to 'Telangana_bus_details.csv'")
    else:
        print("No data scraped.")

Error navigating to page 2: Message: element click intercepted: Element is not clickable at point (631, 2090)
  (Session info: chrome=131.0.6778.69)
Stacktrace:
	GetHandleVerifier [0x00007FF74D3A2775+28773]
	(No symbol) [0x00007FF74D30AFB0]
	(No symbol) [0x00007FF74D1A552A]
	(No symbol) [0x00007FF74D200B5E]
	(No symbol) [0x00007FF74D1FE5FC]
	(No symbol) [0x00007FF74D1FB7E6]
	(No symbol) [0x00007FF74D1FA741]
	(No symbol) [0x00007FF74D1EC970]
	(No symbol) [0x00007FF74D21EF9A]
	(No symbol) [0x00007FF74D1EC1C6]
	(No symbol) [0x00007FF74D21F1B0]
	(No symbol) [0x00007FF74D23F1A4]
	(No symbol) [0x00007FF74D21ED43]
	(No symbol) [0x00007FF74D1EA548]
	(No symbol) [0x00007FF74D1EB6B1]
	GetHandleVerifier [0x00007FF74D6CF45D+3358029]
	GetHandleVerifier [0x00007FF74D6E430D+3443709]
	GetHandleVerifier [0x00007FF74D6D83FD+3394797]
	GetHandleVerifier [0x00007FF74D46929B+842635]
	(No symbol) [0x00007FF74D31654F]
	(No symbol) [0x00007FF74D311FA4]
	(No symbol) [0x00007FF74D31213D]
	(No symbol) [0x00007FF7

2. kerala State

In [2]:
#Import Statements
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd

# Initialize the WebDriver
def initialize_driver():
    driver = webdriver.Chrome() # Opens the Chrome browser.
    driver.maximize_window() # Maximizes the browser window.
    return driver # Returns the driver instance.


# Load a webpage
def load_page(driver, url):
    driver.get(url) # Opens the specified URL in the browser.
    time.sleep(5)  # Waits for 5 seconds to ensure the page loads fully.


# Scrape all bus routes on the current page (or) Scrape Bus Routes
def scrape_bus_routes(driver):
    routes = driver.find_elements(By.CLASS_NAME, 'route')  # Finds all elements with the class `route`.
    route_links = [route.get_attribute('href') for route in routes] # Extracts href links from the elements.
    route_names = [route.text.strip() for route in routes] # Extracts the visible text (route names).
    return zip(route_links, route_names) # Returns a zipped object of links and names.


# Scrape bus details for a specific route
def scrape_bus_details(driver, url, route_name):
    driver.get(url) # Navigates to the route's URL.
    time.sleep(5) # Waits for the page to load.
    
    try:
        # Click "View Buses" button if available
        view_buses = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.CLASS_NAME, "button"))
        )
        view_buses.click()
        time.sleep(5) # Waits for buses to load.
        
        # Scroll to load buses  # Scrolls to the bottom of the page to load additional content.
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(5)
    except:
        print(f"No 'View Buses' button found for {route_name}")
        return [] # Returns an empty list if the button isn't found.
        

    # Collect bus details
    buses = driver.find_elements(By.CLASS_NAME, 'bus-item')  # Finds all bus items on the page.
    bus_details = []
    for bus in buses:  # Iterates through each bus element.
        try:
            bus_details.append({
                "Route_Name": route_name,
                "Route_Link": url,
                "Bus_Name": bus.find_element(By.CLASS_NAME, 'travels').text,  # Extracts bus name.
                "Bus_Type": bus.find_element(By.CLASS_NAME, 'bus-type').text,  # Extracts bus type.
                "Departing_Time": bus.find_element(By.CLASS_NAME, 'dp-time').text,  # Extracts departure time.
                "Duration": bus.find_element(By.CLASS_NAME, 'dur').text,  # Extracts travel duration.
                "Reaching_Time": bus.find_element(By.CLASS_NAME, 'bp-time').text, # Extracts arrival time.
                "Star_Rating": bus.find_element(By.CLASS_NAME, 'rating-sec').text if bus.find_elements(By.CLASS_NAME, 'rating-sec') else '0', # Extracts rating or sets it to '0'.
                "Price": bus.find_element(By.CLASS_NAME, 'fare').text,  # Extracts ticket price.
                "Seat_Availability": bus.find_element(By.XPATH, ".//div[contains(@class, 'seat-left')]").text if bus.find_elements(By.XPATH, ".//div[contains(@class, 'seat-left')]") else '0'
                 # Extracts seat availability or sets it to '0'.
            })
        except Exception as e:
            print(f"Error extracting bus details: {e}")
    return bus_details # Returns a list of dictionaries containing bus details.


# Scrape bus details from all pages
def scrape_all_pages(url, pages=2):
    driver = initialize_driver()  # Initializes the browser.
    all_details = [] # List to store all scraped bus details.

    try:
        for page in range(1, pages + 1): # Loops through pages (default: 2 pages).
            load_page(driver, url) # Loads the main page.
            if page > 1: # If it's not the first page:
                try:
                    # Find and click the pagination tab for the current page.
                    pagination = driver.find_element(By.XPATH, f"//div[contains(@class, 'DC_117_paginationTable') and text()='{page}']")
                    pagination.click()
                    time.sleep(5) # Waits for the next page to load.
                except Exception as e:
                    print(f"Error navigating to page {page}: {e}")
                    continue # Skips to the next iteration if there's an error.
            
            # Scrape routes and details for the current page
            for route_link, route_name in scrape_bus_routes(driver):
                all_details.extend(scrape_bus_details(driver, route_link, route_name))
    finally:
        driver.quit() # Closes the browser at the end of the scraping process.
    return all_details # Returns the complete list of scraped details.


# Main execution
if __name__ == "__main__":
    URL = "https://www.redbus.in/online-booking/ksrtc-kerala/?utm_source=rtchometile"
    bus_details = scrape_all_pages(URL) # Scrapes all pages for bus details.

    # Save the scraped data to a CSV file
    if bus_details:
        pd.DataFrame(bus_details).to_csv('kerala_bus_details.csv', index=False) # Saves data to CSV.
        print("Scraping completed and data saved to 'kerala_bus_details.csv'")
    else:
        print("No data scraped.") #262


No 'View Buses' button found for Kozhikode to Bangalore
Error navigating to page 2: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//div[contains(@class, 'DC_117_paginationTable') and text()='2']"}
  (Session info: chrome=131.0.6778.70); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00007FF7E5736CB5+28821]
	(No symbol) [0x00007FF7E56A3840]
	(No symbol) [0x00007FF7E554578A]
	(No symbol) [0x00007FF7E55991BE]
	(No symbol) [0x00007FF7E55994AC]
	(No symbol) [0x00007FF7E55E2647]
	(No symbol) [0x00007FF7E55BF33F]
	(No symbol) [0x00007FF7E55DF412]
	(No symbol) [0x00007FF7E55BF0A3]
	(No symbol) [0x00007FF7E558A778]
	(No symbol) [0x00007FF7E558B8E1]
	GetHandleVerifier [0x00007FF7E5A6FCAD+3408013]
	GetHandleVerifier [0x00007FF7E5A8741F+3504127]
	GetHandleVerifier [0x00007FF7E5A7B5FD+3455453]
	GetHandleVerifier [0x00007FF7E57FBDBB+8

3. Andhra Pradesh State 

In [23]:
# Import Statements
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd

# Initialize the WebDriver
def initialize_driver():
    driver = webdriver.Chrome()  # Opens the Chrome browser.
    driver.maximize_window()  # Maximizes the browser window.
    return driver  # Returns the driver instance.


# Load a webpage
def load_page(driver, url):
    driver.get(url)  # Opens the specified URL in the browser.
    time.sleep(5)  # Waits for 5 seconds to ensure the page loads fully.


# Scrape all bus routes on the current page (or) Scrape Bus Routes
def scrape_bus_routes(driver):
    routes = driver.find_elements(By.CLASS_NAME, 'route')  # Finds all elements with the class `route`.
    route_links = [route.get_attribute('href') for route in routes]  # Extracts href links from the elements.
    route_names = [route.text.strip() for route in routes]  # Extracts the visible text (route names).
    return zip(route_links, route_names)  # Returns a zipped object of links and names.


# Scrape bus details for a specific route
def scrape_bus_details(driver, url, route_name):
    driver.get(url)  # Navigates to the route's URL.
    time.sleep(5)  # Waits for the page to load.
    
    try:
        # Click "View Buses" button if available
        view_buses = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.CLASS_NAME, "button"))
        )
        view_buses.click()
        time.sleep(5)  # Waits for buses to load.
        
        # Scroll to load buses  # Scrolls to the bottom of the page to load additional content.
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(5)
    except:
        print(f"No 'View Buses' button found for {route_name}")
        return []  # Returns an empty list if the button isn't found.
        

    # Collect bus details
    buses = driver.find_elements(By.CLASS_NAME, 'bus-item')  # Finds all bus items on the page.
    bus_details = []
    for bus in buses:  # Iterates through each bus element.
        try:
            bus_details.append({
                "Route_Name": route_name,
                "Route_Link": url,
                "Bus_Name": bus.find_element(By.CLASS_NAME, 'travels').text,  # Extracts bus name.
                "Bus_Type": bus.find_element(By.CLASS_NAME, 'bus-type').text,  # Extracts bus type.
                "Departing_Time": bus.find_element(By.CLASS_NAME, 'dp-time').text,  # Extracts departure time.
                "Duration": bus.find_element(By.CLASS_NAME, 'dur').text,  # Extracts travel duration.
                "Reaching_Time": bus.find_element(By.CLASS_NAME, 'bp-time').text,  # Extracts arrival time.
                "Star_Rating": bus.find_element(By.CLASS_NAME, 'rating-sec').text if bus.find_elements(By.CLASS_NAME, 'rating-sec') else '0',  # Extracts rating or sets it to '0'.
                "Price": bus.find_element(By.CLASS_NAME, 'fare').text,  # Extracts ticket price.
                "Seat_Availability": bus.find_element(By.XPATH, ".//div[contains(@class, 'seat-left')]").text if bus.find_elements(By.XPATH, ".//div[contains(@class, 'seat-left')]") else '0'
                # Extracts seat availability or sets it to '0'.
            })
        except Exception as e:
            print(f"Error extracting bus details: {e}")
    return bus_details  # Returns a list of dictionaries containing bus details.


# Scrape bus details from all pages
def scrape_all_pages(url, max_pages=6):
    driver = initialize_driver()  # Initializes the browser.
    all_details = []  # List to store all scraped bus details.

    try:
        for i in range(1, max_pages + 1):  # Loops through pages (default: 5 pages).
            load_page(driver, url)  # Loads the main page.
            
            if i > 1:  # If it's not the first page:
                try:
                    # Wait for the pagination element to be loaded
                    WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, '//*[@class = "DC_117_paginationTable"]')))
                    
                    # Find the "next" page button
                    pagination = driver.find_element(By.XPATH, '//*[@class="DC_117_paginationTable"]')
                    next_button = pagination.find_element(By.XPATH, f'.//div[@class="DC_117_pageTabs" and text()="{i}"]')
                    
                    # Click the "next" page button
                    next_button.click()
                    time.sleep(5)  # Wait for the next page to load.
                
                except Exception as e:
                    print(f"Error navigating to page {i}: {e}")
                    continue  # Skip to the next iteration if there's an error.
            
            # Scrape routes and details for the current page
            for route_link, route_name in scrape_bus_routes(driver):
                all_details.extend(scrape_bus_details(driver, route_link, route_name))
    finally:
        driver.quit()  # Closes the browser at the end of the scraping process.

    return all_details  # Returns the complete list of scraped details.


# Main execution
if __name__ == "__main__":
    URL = "https://www.redbus.in/online-booking/apsrtc/?utm_source=rtchometile"
    bus_details = scrape_all_pages(URL)  # Scrapes all pages for bus details.

    # Save the scraped data to a CSV file
    if bus_details:
        pd.DataFrame(bus_details).to_csv('Andhra_bus_details.csv', index=False)  # Saves data to CSV.
        print("Scraping completed and data saved to 'Andhra_bus_details.csv'")
    else:
        print("No data scraped.")  # If no data was scraped


Error navigating to page 2: Message: no such element: Unable to locate element: {"method":"xpath","selector":".//div[@class="DC_117_pageTabs" and text()="2"]"}
  (Session info: chrome=131.0.6778.70); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00007FF7E5736CB5+28821]
	(No symbol) [0x00007FF7E56A3840]
	(No symbol) [0x00007FF7E554578A]
	(No symbol) [0x00007FF7E55991BE]
	(No symbol) [0x00007FF7E55994AC]
	(No symbol) [0x00007FF7E558C52C]
	(No symbol) [0x00007FF7E55BF33F]
	(No symbol) [0x00007FF7E558C3F6]
	(No symbol) [0x00007FF7E55BF510]
	(No symbol) [0x00007FF7E55DF412]
	(No symbol) [0x00007FF7E55BF0A3]
	(No symbol) [0x00007FF7E558A778]
	(No symbol) [0x00007FF7E558B8E1]
	GetHandleVerifier [0x00007FF7E5A6FCAD+3408013]
	GetHandleVerifier [0x00007FF7E5A8741F+3504127]
	GetHandleVerifier [0x00007FF7E5A7B5FD+3455453]
	GetHandleVerifier [0x00007FF7E57FBDBB+835995

4. Kadamba Transport Corporation Limited (KTCL)

In [7]:
#Import Statements
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd

# Initialize the WebDriver
def initialize_driver():
    driver = webdriver.Chrome() # Opens the Chrome browser.
    driver.maximize_window() # Maximizes the browser window.
    return driver # Returns the driver instance.


# Load a webpage
def load_page(driver, url):
    driver.get(url) # Opens the specified URL in the browser.
    time.sleep(5)  # Waits for 5 seconds to ensure the page loads fully.


# Scrape all bus routes on the current page (or) Scrape Bus Routes
def scrape_bus_routes(driver):
    routes = driver.find_elements(By.CLASS_NAME, 'route')  # Finds all elements with the class `route`.
    route_links = [route.get_attribute('href') for route in routes] # Extracts href links from the elements.
    route_names = [route.text.strip() for route in routes] # Extracts the visible text (route names).
    return zip(route_links, route_names) # Returns a zipped object of links and names.


# Scrape bus details for a specific route
def scrape_bus_details(driver, url, route_name):
    driver.get(url) # Navigates to the route's URL.
    time.sleep(5) # Waits for the page to load.
    
    try:
        # Click "View Buses" button if available
        view_buses = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.CLASS_NAME, "button"))
        )
        view_buses.click()
        time.sleep(5) # Waits for buses to load.
        
        # Scroll to load buses  # Scrolls to the bottom of the page to load additional content.
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(5)
    except:
        print(f"No 'View Buses' button found for {route_name}")
        return [] # Returns an empty list if the button isn't found.
        

    # Collect bus details
    buses = driver.find_elements(By.CLASS_NAME, 'bus-item')  # Finds all bus items on the page.
    bus_details = []
    for bus in buses:  # Iterates through each bus element.
        try:
            bus_details.append({
                "Route_Name": route_name,
                "Route_Link": url,
                "Bus_Name": bus.find_element(By.CLASS_NAME, 'travels').text,  # Extracts bus name.
                "Bus_Type": bus.find_element(By.CLASS_NAME, 'bus-type').text,  # Extracts bus type.
                "Departing_Time": bus.find_element(By.CLASS_NAME, 'dp-time').text,  # Extracts departure time.
                "Duration": bus.find_element(By.CLASS_NAME, 'dur').text,  # Extracts travel duration.
                "Reaching_Time": bus.find_element(By.CLASS_NAME, 'bp-time').text, # Extracts arrival time.
                "Star_Rating": bus.find_element(By.CLASS_NAME, 'rating-sec').text if bus.find_elements(By.CLASS_NAME, 'rating-sec') else '0', # Extracts rating or sets it to '0'.
                "Price": bus.find_element(By.CLASS_NAME, 'fare').text,  # Extracts ticket price.
                "Seat_Availability": bus.find_element(By.XPATH, ".//div[contains(@class, 'seat-left')]").text if bus.find_elements(By.XPATH, ".//div[contains(@class, 'seat-left')]") else '0'
                 # Extracts seat availability or sets it to '0'.
            })
        except Exception as e:
            print(f"Error extracting bus details: {e}")
    return bus_details # Returns a list of dictionaries containing bus details.


# Scrape bus details from all pages
def scrape_all_pages(url, pages=5):
    driver = initialize_driver()  # Initializes the browser.
    all_details = [] # List to store all scraped bus details.

    try:
        for page in range(1, pages + 1): # Loops through pages (default: 5 pages).
            load_page(driver, url) # Loads the main page.
            if page > 1: # If it's not the first page:
                try:
                    # Find and click the pagination tab for the current page.
                    pagination = driver.find_element(By.XPATH, f"//div[contains(@class, 'DC_117_paginationTable') and text()='{page}']")
                    pagination.click()
                    time.sleep(5) # Waits for the next page to load.
                except Exception as e:
                    print(f"Error navigating to page {page}: {e}")
                    continue # Skips to the next iteration if there's an error.
            
            # Scrape routes and details for the current page
            for route_link, route_name in scrape_bus_routes(driver):
                all_details.extend(scrape_bus_details(driver, route_link, route_name))
    finally:
        driver.quit() # Closes the browser at the end of the scraping process.
    return all_details # Returns the complete list of scraped details.


# Main execution
if __name__ == "__main__":
    URL = "https://www.redbus.in/online-booking/ktcl/?utm_source=rtchometile"
    bus_details = scrape_all_pages(URL) # Scrapes all pages for bus details.

    # Save the scraped data to a CSV file
    if bus_details:
        pd.DataFrame(bus_details).to_csv('Kadamba_bus_details.csv', index=False) # Saves data to CSV.
        print("Scraping completed and data saved to 'Kadamba_bus_details.csv'")
    else:
        print("No data scraped.")


No 'View Buses' button found for Goa to Pune
Error navigating to page 2: Message: element click intercepted: Element is not clickable at point (591, 2090)
  (Session info: chrome=131.0.6778.69)
Stacktrace:
	GetHandleVerifier [0x00007FF74D3A2775+28773]
	(No symbol) [0x00007FF74D30AFB0]
	(No symbol) [0x00007FF74D1A552A]
	(No symbol) [0x00007FF74D200B5E]
	(No symbol) [0x00007FF74D1FE5FC]
	(No symbol) [0x00007FF74D1FB7E6]
	(No symbol) [0x00007FF74D1FA741]
	(No symbol) [0x00007FF74D1EC970]
	(No symbol) [0x00007FF74D21EF9A]
	(No symbol) [0x00007FF74D1EC1C6]
	(No symbol) [0x00007FF74D21F1B0]
	(No symbol) [0x00007FF74D23F1A4]
	(No symbol) [0x00007FF74D21ED43]
	(No symbol) [0x00007FF74D1EA548]
	(No symbol) [0x00007FF74D1EB6B1]
	GetHandleVerifier [0x00007FF74D6CF45D+3358029]
	GetHandleVerifier [0x00007FF74D6E430D+3443709]
	GetHandleVerifier [0x00007FF74D6D83FD+3394797]
	GetHandleVerifier [0x00007FF74D46929B+842635]
	(No symbol) [0x00007FF74D31654F]
	(No symbol) [0x00007FF74D311FA4]
	(No symbol) 

5. Rajasthan State

In [8]:
#Import Statements
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd

# Initialize the WebDriver
def initialize_driver():
    driver = webdriver.Chrome() # Opens the Chrome browser.
    driver.maximize_window() # Maximizes the browser window.
    return driver # Returns the driver instance.


# Load a webpage
def load_page(driver, url):
    driver.get(url) # Opens the specified URL in the browser.
    time.sleep(5)  # Waits for 5 seconds to ensure the page loads fully.


# Scrape all bus routes on the current page (or) Scrape Bus Routes
def scrape_bus_routes(driver):
    routes = driver.find_elements(By.CLASS_NAME, 'route')  # Finds all elements with the class `route`.
    route_links = [route.get_attribute('href') for route in routes] # Extracts href links from the elements.
    route_names = [route.text.strip() for route in routes] # Extracts the visible text (route names).
    return zip(route_links, route_names) # Returns a zipped object of links and names.


# Scrape bus details for a specific route
def scrape_bus_details(driver, url, route_name):
    driver.get(url) # Navigates to the route's URL.
    time.sleep(5) # Waits for the page to load.
    
    try:
        # Click "View Buses" button if available
        view_buses = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.CLASS_NAME, "button"))
        )
        view_buses.click()
        time.sleep(5) # Waits for buses to load.
        
        # Scroll to load buses  # Scrolls to the bottom of the page to load additional content.
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(5)
    except:
        print(f"No 'View Buses' button found for {route_name}")
        return [] # Returns an empty list if the button isn't found.
        

    # Collect bus details
    buses = driver.find_elements(By.CLASS_NAME, 'bus-item')  # Finds all bus items on the page.
    bus_details = []
    for bus in buses:  # Iterates through each bus element.
        try:
            bus_details.append({
                "Route_Name": route_name,
                "Route_Link": url,
                "Bus_Name": bus.find_element(By.CLASS_NAME, 'travels').text,  # Extracts bus name.
                "Bus_Type": bus.find_element(By.CLASS_NAME, 'bus-type').text,  # Extracts bus type.
                "Departing_Time": bus.find_element(By.CLASS_NAME, 'dp-time').text,  # Extracts departure time.
                "Duration": bus.find_element(By.CLASS_NAME, 'dur').text,  # Extracts travel duration.
                "Reaching_Time": bus.find_element(By.CLASS_NAME, 'bp-time').text, # Extracts arrival time.
                "Star_Rating": bus.find_element(By.CLASS_NAME, 'rating-sec').text if bus.find_elements(By.CLASS_NAME, 'rating-sec') else '0', # Extracts rating or sets it to '0'.
                "Price": bus.find_element(By.CLASS_NAME, 'fare').text,  # Extracts ticket price.
                "Seat_Availability": bus.find_element(By.XPATH, ".//div[contains(@class, 'seat-left')]").text if bus.find_elements(By.XPATH, ".//div[contains(@class, 'seat-left')]") else '0'
                 # Extracts seat availability or sets it to '0'.
            })
        except Exception as e:
            print(f"Error extracting bus details: {e}")
    return bus_details # Returns a list of dictionaries containing bus details.


# Scrape bus details from all pages
def scrape_all_pages(url, pages=3):
    driver = initialize_driver()  # Initializes the browser.
    all_details = [] # List to store all scraped bus details.

    try:
        for page in range(1, pages + 1): # Loops through pages (default: 3 pages).
            load_page(driver, url) # Loads the main page.
            if page > 1: # If it's not the first page:
                try:
                    # Find and click the pagination tab for the current page.
                    pagination = driver.find_element(By.XPATH, f"//div[contains(@class, 'DC_117_pageTabs') and text()='{page}']")
                    pagination.click()
                    time.sleep(5) # Waits for the next page to load.
                except Exception as e:
                    print(f"Error navigating to page {page}: {e}")
                    continue # Skips to the next iteration if there's an error.
            
            # Scrape routes and details for the current page
            for route_link, route_name in scrape_bus_routes(driver):
                all_details.extend(scrape_bus_details(driver, route_link, route_name))
    finally:
        driver.quit() # Closes the browser at the end of the scraping process.
    return all_details # Returns the complete list of scraped details.


# Main execution
if __name__ == "__main__":
    URL = "https://www.redbus.in/online-booking/rsrtc/?utm_source=rtchometile"
    bus_details = scrape_all_pages(URL) # Scrapes all pages for bus details.

    # Save the scraped data to a CSV file
    if bus_details:
        pd.DataFrame(bus_details).to_csv('Rajasthan_bus_details.csv', index=False) # Saves data to CSV.
        print("Scraping completed and data saved to 'Rajasthan_bus_details.csv'")
    else:
        print("No data scraped.")


No 'View Buses' button found for Udaipur to Jodhpur
Error navigating to page 2: Message: element click intercepted: Element is not clickable at point (651, 2092)
  (Session info: chrome=131.0.6778.69)
Stacktrace:
	GetHandleVerifier [0x00007FF74D3A2775+28773]
	(No symbol) [0x00007FF74D30AFB0]
	(No symbol) [0x00007FF74D1A552A]
	(No symbol) [0x00007FF74D200B5E]
	(No symbol) [0x00007FF74D1FE5FC]
	(No symbol) [0x00007FF74D1FB7E6]
	(No symbol) [0x00007FF74D1FA741]
	(No symbol) [0x00007FF74D1EC970]
	(No symbol) [0x00007FF74D21EF9A]
	(No symbol) [0x00007FF74D1EC1C6]
	(No symbol) [0x00007FF74D21F1B0]
	(No symbol) [0x00007FF74D23F1A4]
	(No symbol) [0x00007FF74D21ED43]
	(No symbol) [0x00007FF74D1EA548]
	(No symbol) [0x00007FF74D1EB6B1]
	GetHandleVerifier [0x00007FF74D6CF45D+3358029]
	GetHandleVerifier [0x00007FF74D6E430D+3443709]
	GetHandleVerifier [0x00007FF74D6D83FD+3394797]
	GetHandleVerifier [0x00007FF74D46929B+842635]
	(No symbol) [0x00007FF74D31654F]
	(No symbol) [0x00007FF74D311FA4]
	(No s

6. South Bengal State

In [9]:
#Import Statements
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd

# Initialize the WebDriver
def initialize_driver():
    driver = webdriver.Chrome() # Opens the Chrome browser.
    driver.maximize_window() # Maximizes the browser window.
    return driver # Returns the driver instance.


# Load a webpage
def load_page(driver, url):
    driver.get(url) # Opens the specified URL in the browser.
    time.sleep(5)  # Waits for 5 seconds to ensure the page loads fully.


# Scrape all bus routes on the current page (or) Scrape Bus Routes
def scrape_bus_routes(driver):
    routes = driver.find_elements(By.CLASS_NAME, 'route')  # Finds all elements with the class `route`.
    route_links = [route.get_attribute('href') for route in routes] # Extracts href links from the elements.
    route_names = [route.text.strip() for route in routes] # Extracts the visible text (route names).
    return zip(route_links, route_names) # Returns a zipped object of links and names.


# Scrape bus details for a specific route
def scrape_bus_details(driver, url, route_name):
    driver.get(url) # Navigates to the route's URL.
    time.sleep(5) # Waits for the page to load.
    
    try:
        # Click "View Buses" button if available
        view_buses = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.CLASS_NAME, "button"))
        )
        view_buses.click()
        time.sleep(5) # Waits for buses to load.
        
        # Scroll to load buses  # Scrolls to the bottom of the page to load additional content.
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(5)
    except:
        print(f"No 'View Buses' button found for {route_name}")
        return [] # Returns an empty list if the button isn't found.
        

    # Collect bus details
    buses = driver.find_elements(By.CLASS_NAME, 'bus-item')  # Finds all bus items on the page.
    bus_details = []
    for bus in buses:  # Iterates through each bus element.
        try:
            bus_details.append({
                "Route_Name": route_name,
                "Route_Link": url,
                "Bus_Name": bus.find_element(By.CLASS_NAME, 'travels').text,  # Extracts bus name.
                "Bus_Type": bus.find_element(By.CLASS_NAME, 'bus-type').text,  # Extracts bus type.
                "Departing_Time": bus.find_element(By.CLASS_NAME, 'dp-time').text,  # Extracts departure time.
                "Duration": bus.find_element(By.CLASS_NAME, 'dur').text,  # Extracts travel duration.
                "Reaching_Time": bus.find_element(By.CLASS_NAME, 'bp-time').text, # Extracts arrival time.
                "Star_Rating": bus.find_element(By.CLASS_NAME, 'rating-sec').text if bus.find_elements(By.CLASS_NAME, 'rating-sec') else '0', # Extracts rating or sets it to '0'.
                "Price": bus.find_element(By.CLASS_NAME, 'fare').text,  # Extracts ticket price.
                "Seat_Availability": bus.find_element(By.XPATH, ".//div[contains(@class, 'seat-left')]").text if bus.find_elements(By.XPATH, ".//div[contains(@class, 'seat-left')]") else '0'
                 # Extracts seat availability or sets it to '0'.
            })
        except Exception as e:
            print(f"Error extracting bus details: {e}")
    return bus_details # Returns a list of dictionaries containing bus details.


# Scrape bus details from all pages
def scrape_all_pages(url, pages=3):
    driver = initialize_driver()  # Initializes the browser.
    all_details = [] # List to store all scraped bus details.

    try:
        for page in range(1, pages + 1): # Loops through pages (default: 3 pages).
            load_page(driver, url) # Loads the main page.
            if page > 1: # If it's not the first page:
                try:
                    # Find and click the pagination tab for the current page.
                    pagination = driver.find_element(By.XPATH, f"//div[contains(@class, 'DC_117_pageTabs') and text()='{page}']")
                    pagination.click()
                    time.sleep(5) # Waits for the next page to load.
                except Exception as e:
                    print(f"Error navigating to page {page}: {e}")
                    continue # Skips to the next iteration if there's an error.
            
            # Scrape routes and details for the current page
            for route_link, route_name in scrape_bus_routes(driver):
                all_details.extend(scrape_bus_details(driver, route_link, route_name))
    finally:
        driver.quit() # Closes the browser at the end of the scraping process.
    return all_details # Returns the complete list of scraped details.


# Main execution
if __name__ == "__main__":
    URL = "https://www.redbus.in/online-booking/south-bengal-state-transport-corporation-sbstc/?utm_source=rtchometile"
    bus_details = scrape_all_pages(URL) # Scrapes all pages for bus details.

    # Save the scraped data to a CSV file
    if bus_details:
        pd.DataFrame(bus_details).to_csv('SBengal_bus_details.csv', index=False) # Saves data to CSV.
        print("Scraping completed and data saved to 'SBengal_bus_details.csv'")
    else:
        print("No data scraped.")


Error navigating to page 2: Message: element click intercepted: Element is not clickable at point (591, 2085)
  (Session info: chrome=131.0.6778.69)
Stacktrace:
	GetHandleVerifier [0x00007FF73D342775+28773]
	(No symbol) [0x00007FF73D2AAFB0]
	(No symbol) [0x00007FF73D14552A]
	(No symbol) [0x00007FF73D1A0B5E]
	(No symbol) [0x00007FF73D19E5FC]
	(No symbol) [0x00007FF73D19B7E6]
	(No symbol) [0x00007FF73D19A741]
	(No symbol) [0x00007FF73D18C970]
	(No symbol) [0x00007FF73D1BEF9A]
	(No symbol) [0x00007FF73D18C1C6]
	(No symbol) [0x00007FF73D1BF1B0]
	(No symbol) [0x00007FF73D1DF1A4]
	(No symbol) [0x00007FF73D1BED43]
	(No symbol) [0x00007FF73D18A548]
	(No symbol) [0x00007FF73D18B6B1]
	GetHandleVerifier [0x00007FF73D66F45D+3358029]
	GetHandleVerifier [0x00007FF73D68430D+3443709]
	GetHandleVerifier [0x00007FF73D6783FD+3394797]
	GetHandleVerifier [0x00007FF73D40929B+842635]
	(No symbol) [0x00007FF73D2B654F]
	(No symbol) [0x00007FF73D2B1FA4]
	(No symbol) [0x00007FF73D2B213D]
	(No symbol) [0x00007FF7

7. Himachal State 

In [10]:
#Import Statements
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd

# Initialize the WebDriver
def initialize_driver():
    driver = webdriver.Chrome() # Opens the Chrome browser.
    driver.maximize_window() # Maximizes the browser window.
    return driver # Returns the driver instance.


# Load a webpage
def load_page(driver, url):
    driver.get(url) # Opens the specified URL in the browser.
    time.sleep(5)  # Waits for 5 seconds to ensure the page loads fully.


# Scrape all bus routes on the current page (or) Scrape Bus Routes
def scrape_bus_routes(driver):
    routes = driver.find_elements(By.CLASS_NAME, 'route')  # Finds all elements with the class `route`.
    route_links = [route.get_attribute('href') for route in routes] # Extracts href links from the elements.
    route_names = [route.text.strip() for route in routes] # Extracts the visible text (route names).
    return zip(route_links, route_names) # Returns a zipped object of links and names.


# Scrape bus details for a specific route
def scrape_bus_details(driver, url, route_name):
    driver.get(url) # Navigates to the route's URL.
    time.sleep(5) # Waits for the page to load.
    
    try:
        # Click "View Buses" button if available
        view_buses = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.CLASS_NAME, "button"))
        )
        view_buses.click()
        time.sleep(5) # Waits for buses to load.
        
        # Scroll to load buses  # Scrolls to the bottom of the page to load additional content.
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(5)
    except:
        print(f"No 'View Buses' button found for {route_name}")
        return [] # Returns an empty list if the button isn't found.
        

    # Collect bus details
    buses = driver.find_elements(By.CLASS_NAME, 'bus-item')  # Finds all bus items on the page.
    bus_details = []
    for bus in buses:  # Iterates through each bus element.
        try:
            bus_details.append({
                "Route_Name": route_name,
                "Route_Link": url,
                "Bus_Name": bus.find_element(By.CLASS_NAME, 'travels').text,  # Extracts bus name.
                "Bus_Type": bus.find_element(By.CLASS_NAME, 'bus-type').text,  # Extracts bus type.
                "Departing_Time": bus.find_element(By.CLASS_NAME, 'dp-time').text,  # Extracts departure time.
                "Duration": bus.find_element(By.CLASS_NAME, 'dur').text,  # Extracts travel duration.
                "Reaching_Time": bus.find_element(By.CLASS_NAME, 'bp-time').text, # Extracts arrival time.
                "Star_Rating": bus.find_element(By.CLASS_NAME, 'rating-sec').text if bus.find_elements(By.CLASS_NAME, 'rating-sec') else '0', # Extracts rating or sets it to '0'.
                "Price": bus.find_element(By.CLASS_NAME, 'fare').text,  # Extracts ticket price.
                "Seat_Availability": bus.find_element(By.XPATH, ".//div[contains(@class, 'seat-left')]").text if bus.find_elements(By.XPATH, ".//div[contains(@class, 'seat-left')]") else '0'
                 # Extracts seat availability or sets it to '0'.
            })
        except Exception as e:
            print(f"Error extracting bus details: {e}")
    return bus_details # Returns a list of dictionaries containing bus details.


# Scrape bus details from all pages
def scrape_all_pages(url, pages=3):
    driver = initialize_driver()  # Initializes the browser.
    all_details = [] # List to store all scraped bus details.

    try:
        for page in range(1, pages + 1): # Loops through pages (default: 3 pages).
            load_page(driver, url) # Loads the main page.
            if page > 1: # If it's not the first page:
                try:
                    # Find and click the pagination tab for the current page.
                    pagination = driver.find_element(By.XPATH, f"//div[contains(@class, 'DC_117_pageTabs') and text()='{page}']")
                    pagination.click()
                    time.sleep(5) # Waits for the next page to load.
                except Exception as e:
                    print(f"Error navigating to page {page}: {e}")
                    continue # Skips to the next iteration if there's an error.
            
            # Scrape routes and details for the current page
            for route_link, route_name in scrape_bus_routes(driver):
                all_details.extend(scrape_bus_details(driver, route_link, route_name))
    finally:
        driver.quit() # Closes the browser at the end of the scraping process.
    return all_details # Returns the complete list of scraped details.


# Main execution
if __name__ == "__main__":
    URL = "https://www.redbus.in/online-booking/hrtc/?utm_source=rtchometile"
    bus_details = scrape_all_pages(URL) # Scrapes all pages for bus details.

    # Save the scraped data to a CSV file
    if bus_details:
        pd.DataFrame(bus_details).to_csv('Himachal_bus_details.csv', index=False) # Saves data to CSV.
        print("Scraping completed and data saved to 'Himachal_bus_details.csv'")
    else:
        print("No data scraped.")


Error navigating to page 2: Message: element click intercepted: Element is not clickable at point (611, 2092)
  (Session info: chrome=131.0.6778.69)
Stacktrace:
	GetHandleVerifier [0x00007FF73D342775+28773]
	(No symbol) [0x00007FF73D2AAFB0]
	(No symbol) [0x00007FF73D14552A]
	(No symbol) [0x00007FF73D1A0B5E]
	(No symbol) [0x00007FF73D19E5FC]
	(No symbol) [0x00007FF73D19B7E6]
	(No symbol) [0x00007FF73D19A741]
	(No symbol) [0x00007FF73D18C970]
	(No symbol) [0x00007FF73D1BEF9A]
	(No symbol) [0x00007FF73D18C1C6]
	(No symbol) [0x00007FF73D1BF1B0]
	(No symbol) [0x00007FF73D1DF1A4]
	(No symbol) [0x00007FF73D1BED43]
	(No symbol) [0x00007FF73D18A548]
	(No symbol) [0x00007FF73D18B6B1]
	GetHandleVerifier [0x00007FF73D66F45D+3358029]
	GetHandleVerifier [0x00007FF73D68430D+3443709]
	GetHandleVerifier [0x00007FF73D6783FD+3394797]
	GetHandleVerifier [0x00007FF73D40929B+842635]
	(No symbol) [0x00007FF73D2B654F]
	(No symbol) [0x00007FF73D2B1FA4]
	(No symbol) [0x00007FF73D2B213D]
	(No symbol) [0x00007FF7

8. Assam State 

In [11]:
#Import Statements
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd

# Initialize the WebDriver
def initialize_driver():
    driver = webdriver.Chrome() # Opens the Chrome browser.
    driver.maximize_window() # Maximizes the browser window.
    return driver # Returns the driver instance.


# Load a webpage
def load_page(driver, url):
    driver.get(url) # Opens the specified URL in the browser.
    time.sleep(5)  # Waits for 5 seconds to ensure the page loads fully.


# Scrape all bus routes on the current page (or) Scrape Bus Routes
def scrape_bus_routes(driver):
    routes = driver.find_elements(By.CLASS_NAME, 'route')  # Finds all elements with the class `route`.
    route_links = [route.get_attribute('href') for route in routes] # Extracts href links from the elements.
    route_names = [route.text.strip() for route in routes] # Extracts the visible text (route names).
    return zip(route_links, route_names) # Returns a zipped object of links and names.


# Scrape bus details for a specific route
def scrape_bus_details(driver, url, route_name):
    driver.get(url) # Navigates to the route's URL.
    time.sleep(5) # Waits for the page to load.
    
    try:
        # Click "View Buses" button if available
        view_buses = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.CLASS_NAME, "button"))
        )
        view_buses.click()
        time.sleep(5) # Waits for buses to load.
        
        # Scroll to load buses  # Scrolls to the bottom of the page to load additional content.
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(5)
    except:
        print(f"No 'View Buses' button found for {route_name}")
        return [] # Returns an empty list if the button isn't found.
        

    # Collect bus details
    buses = driver.find_elements(By.CLASS_NAME, 'bus-item')  # Finds all bus items on the page.
    bus_details = []
    for bus in buses:  # Iterates through each bus element.
        try:
            bus_details.append({
                "Route_Name": route_name,
                "Route_Link": url,
                "Bus_Name": bus.find_element(By.CLASS_NAME, 'travels').text,  # Extracts bus name.
                "Bus_Type": bus.find_element(By.CLASS_NAME, 'bus-type').text,  # Extracts bus type.
                "Departing_Time": bus.find_element(By.CLASS_NAME, 'dp-time').text,  # Extracts departure time.
                "Duration": bus.find_element(By.CLASS_NAME, 'dur').text,  # Extracts travel duration.
                "Reaching_Time": bus.find_element(By.CLASS_NAME, 'bp-time').text, # Extracts arrival time.
                "Star_Rating": bus.find_element(By.CLASS_NAME, 'rating-sec').text if bus.find_elements(By.CLASS_NAME, 'rating-sec') else '0', # Extracts rating or sets it to '0'.
                "Price": bus.find_element(By.CLASS_NAME, 'fare').text,  # Extracts ticket price.
                "Seat_Availability": bus.find_element(By.XPATH, ".//div[contains(@class, 'seat-left')]").text if bus.find_elements(By.XPATH, ".//div[contains(@class, 'seat-left')]") else '0'
                 # Extracts seat availability or sets it to '0'.
            })
        except Exception as e:
            print(f"Error extracting bus details: {e}")
    return bus_details # Returns a list of dictionaries containing bus details.


# Scrape bus details from all pages
def scrape_all_pages(url, pages=3):
    driver = initialize_driver()  # Initializes the browser.
    all_details = [] # List to store all scraped bus details.

    try:
        for page in range(1, pages + 1): # Loops through pages (default: 3 pages).
            load_page(driver, url) # Loads the main page.
            if page > 1: # If it's not the first page:
                try:
                    # Find and click the pagination tab for the current page.
                    pagination = driver.find_element(By.XPATH, f"//div[contains(@class, 'DC_117_pageTabs') and text()='{page}']")
                    pagination.click()
                    time.sleep(5) # Waits for the next page to load.
                except Exception as e:
                    print(f"Error navigating to page {page}: {e}")
                    continue # Skips to the next iteration if there's an error.
            
            # Scrape routes and details for the current page
            for route_link, route_name in scrape_bus_routes(driver):
                all_details.extend(scrape_bus_details(driver, route_link, route_name))
    finally:
        driver.quit() # Closes the browser at the end of the scraping process.
    return all_details # Returns the complete list of scraped details.


# Main execution
if __name__ == "__main__":
    URL = "https://www.redbus.in/online-booking/astc/?utm_source=rtchometile"
    bus_details = scrape_all_pages(URL) # Scrapes all pages for bus details.

    # Save the scraped data to a CSV file
    if bus_details:
        pd.DataFrame(bus_details).to_csv('Assam_bus_details.csv', index=False) # Saves data to CSV.
        print("Scraping completed and data saved to 'Assam_bus_details.csv'")
    else:
        print("No data scraped.")


Error navigating to page 2: Message: element click intercepted: Element is not clickable at point (591, 2090)
  (Session info: chrome=131.0.6778.69)
Stacktrace:
	GetHandleVerifier [0x00007FF73D342775+28773]
	(No symbol) [0x00007FF73D2AAFB0]
	(No symbol) [0x00007FF73D14552A]
	(No symbol) [0x00007FF73D1A0B5E]
	(No symbol) [0x00007FF73D19E5FC]
	(No symbol) [0x00007FF73D19B7E6]
	(No symbol) [0x00007FF73D19A741]
	(No symbol) [0x00007FF73D18C970]
	(No symbol) [0x00007FF73D1BEF9A]
	(No symbol) [0x00007FF73D18C1C6]
	(No symbol) [0x00007FF73D1BF1B0]
	(No symbol) [0x00007FF73D1DF1A4]
	(No symbol) [0x00007FF73D1BED43]
	(No symbol) [0x00007FF73D18A548]
	(No symbol) [0x00007FF73D18B6B1]
	GetHandleVerifier [0x00007FF73D66F45D+3358029]
	GetHandleVerifier [0x00007FF73D68430D+3443709]
	GetHandleVerifier [0x00007FF73D6783FD+3394797]
	GetHandleVerifier [0x00007FF73D40929B+842635]
	(No symbol) [0x00007FF73D2B654F]
	(No symbol) [0x00007FF73D2B1FA4]
	(No symbol) [0x00007FF73D2B213D]
	(No symbol) [0x00007FF7

9. Jammu State

In [12]:
#Import Statements
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd

# Initialize the WebDriver
def initialize_driver():
    driver = webdriver.Chrome() # Opens the Chrome browser.
    driver.maximize_window() # Maximizes the browser window.
    return driver # Returns the driver instance.


# Load a webpage
def load_page(driver, url):
    driver.get(url) # Opens the specified URL in the browser.
    time.sleep(5)  # Waits for 5 seconds to ensure the page loads fully.


# Scrape all bus routes on the current page (or) Scrape Bus Routes
def scrape_bus_routes(driver):
    routes = driver.find_elements(By.CLASS_NAME, 'route')  # Finds all elements with the class `route`.
    route_links = [route.get_attribute('href') for route in routes] # Extracts href links from the elements.
    route_names = [route.text.strip() for route in routes] # Extracts the visible text (route names).
    return zip(route_links, route_names) # Returns a zipped object of links and names.


# Scrape bus details for a specific route
def scrape_bus_details(driver, url, route_name):
    driver.get(url) # Navigates to the route's URL.
    time.sleep(5) # Waits for the page to load.
    
    try:
        # Click "View Buses" button if available
        view_buses = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.CLASS_NAME, "button"))
        )
        view_buses.click()
        time.sleep(5) # Waits for buses to load.
        
        # Scroll to load buses  # Scrolls to the bottom of the page to load additional content.
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(5)
    except:
        print(f"No 'View Buses' button found for {route_name}")
        return [] # Returns an empty list if the button isn't found.
        

    # Collect bus details
    buses = driver.find_elements(By.CLASS_NAME, 'bus-item')  # Finds all bus items on the page.
    bus_details = []
    for bus in buses:  # Iterates through each bus element.
        try:
            bus_details.append({
                "Route_Name": route_name,
                "Route_Link": url,
                "Bus_Name": bus.find_element(By.CLASS_NAME, 'travels').text,  # Extracts bus name.
                "Bus_Type": bus.find_element(By.CLASS_NAME, 'bus-type').text,  # Extracts bus type.
                "Departing_Time": bus.find_element(By.CLASS_NAME, 'dp-time').text,  # Extracts departure time.
                "Duration": bus.find_element(By.CLASS_NAME, 'dur').text,  # Extracts travel duration.
                "Reaching_Time": bus.find_element(By.CLASS_NAME, 'bp-time').text, # Extracts arrival time.
                "Star_Rating": bus.find_element(By.CLASS_NAME, 'rating-sec').text if bus.find_elements(By.CLASS_NAME, 'rating-sec') else '0', # Extracts rating or sets it to '0'.
                "Price": bus.find_element(By.CLASS_NAME, 'fare').text,  # Extracts ticket price.
                "Seat_Availability": bus.find_element(By.XPATH, ".//div[contains(@class, 'seat-left')]").text if bus.find_elements(By.XPATH, ".//div[contains(@class, 'seat-left')]") else '0'
                 # Extracts seat availability or sets it to '0'.
            })
        except Exception as e:
            print(f"Error extracting bus details: {e}")
    return bus_details # Returns a list of dictionaries containing bus details.


# Scrape bus details from all pages
def scrape_all_pages(url, pages=3):
    driver = initialize_driver()  # Initializes the browser.
    all_details = [] # List to store all scraped bus details.

    try:
        for page in range(1, pages + 1): # Loops through pages (default: 3 pages).
            load_page(driver, url) # Loads the main page.
            if page > 1: # If it's not the first page:
                try:
                    # Find and click the pagination tab for the current page.
                    pagination = driver.find_element(By.XPATH, f"//div[contains(@class, 'DC_117_pageTabs') and text()='{page}']")
                    pagination.click()
                    time.sleep(5) # Waits for the next page to load.
                except Exception as e:
                    print(f"Error navigating to page {page}: {e}")
                    continue # Skips to the next iteration if there's an error.
            
            # Scrape routes and details for the current page
            for route_link, route_name in scrape_bus_routes(driver):
                all_details.extend(scrape_bus_details(driver, route_link, route_name))
    finally:
        driver.quit() # Closes the browser at the end of the scraping process.
    return all_details # Returns the complete list of scraped details.


# Main execution
if __name__ == "__main__":
    URL = "https://www.redbus.in/online-booking/jksrtc"
    bus_details = scrape_all_pages(URL) # Scrapes all pages for bus details.

    # Save the scraped data to a CSV file
    if bus_details:
        pd.DataFrame(bus_details).to_csv('Jammu_bus_details.csv', index=False) # Saves data to CSV.
        print("Scraping completed and data saved to 'Jammu_bus_details.csv'")
    else:
        print("No data scraped.")


Error navigating to page 2: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//div[contains(@class, 'DC_117_pageTabs') and text()='2']"}
  (Session info: chrome=131.0.6778.69); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00007FF73D342775+28773]
	(No symbol) [0x00007FF73D2AAFB0]
	(No symbol) [0x00007FF73D14552A]
	(No symbol) [0x00007FF73D198EAE]
	(No symbol) [0x00007FF73D19919C]
	(No symbol) [0x00007FF73D1E23F7]
	(No symbol) [0x00007FF73D1BEFDF]
	(No symbol) [0x00007FF73D1DF1A4]
	(No symbol) [0x00007FF73D1BED43]
	(No symbol) [0x00007FF73D18A548]
	(No symbol) [0x00007FF73D18B6B1]
	GetHandleVerifier [0x00007FF73D66F45D+3358029]
	GetHandleVerifier [0x00007FF73D68430D+3443709]
	GetHandleVerifier [0x00007FF73D6783FD+3394797]
	GetHandleVerifier [0x00007FF73D40929B+842635]
	(No symbol) [0x00007FF73D2B654F]
	(No symbol) [0x00007F

10. West Bengal State

In [13]:
#Import Statements
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd

# Initialize the WebDriver
def initialize_driver():
    driver = webdriver.Chrome() # Opens the Chrome browser.
    driver.maximize_window() # Maximizes the browser window.
    return driver # Returns the driver instance.


# Load a webpage
def load_page(driver, url):
    driver.get(url) # Opens the specified URL in the browser.
    time.sleep(5)  # Waits for 5 seconds to ensure the page loads fully.


# Scrape all bus routes on the current page (or) Scrape Bus Routes
def scrape_bus_routes(driver):
    routes = driver.find_elements(By.CLASS_NAME, 'route')  # Finds all elements with the class `route`.
    route_links = [route.get_attribute('href') for route in routes] # Extracts href links from the elements.
    route_names = [route.text.strip() for route in routes] # Extracts the visible text (route names).
    return zip(route_links, route_names) # Returns a zipped object of links and names.


# Scrape bus details for a specific route
def scrape_bus_details(driver, url, route_name):
    driver.get(url) # Navigates to the route's URL.
    time.sleep(5) # Waits for the page to load.
    
    try:
        # Click "View Buses" button if available
        view_buses = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.CLASS_NAME, "button"))
        )
        view_buses.click()
        time.sleep(5) # Waits for buses to load.
        
        # Scroll to load buses  # Scrolls to the bottom of the page to load additional content.
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(5)
    except:
        print(f"No 'View Buses' button found for {route_name}")
        return [] # Returns an empty list if the button isn't found.
        

    # Collect bus details
    buses = driver.find_elements(By.CLASS_NAME, 'bus-item')  # Finds all bus items on the page.
    bus_details = []
    for bus in buses:  # Iterates through each bus element.
        try:
            bus_details.append({
                "Route_Name": route_name,
                "Route_Link": url,
                "Bus_Name": bus.find_element(By.CLASS_NAME, 'travels').text,  # Extracts bus name.
                "Bus_Type": bus.find_element(By.CLASS_NAME, 'bus-type').text,  # Extracts bus type.
                "Departing_Time": bus.find_element(By.CLASS_NAME, 'dp-time').text,  # Extracts departure time.
                "Duration": bus.find_element(By.CLASS_NAME, 'dur').text,  # Extracts travel duration.
                "Reaching_Time": bus.find_element(By.CLASS_NAME, 'bp-time').text, # Extracts arrival time.
                "Star_Rating": bus.find_element(By.CLASS_NAME, 'rating-sec').text if bus.find_elements(By.CLASS_NAME, 'rating-sec') else '0', # Extracts rating or sets it to '0'.
                "Price": bus.find_element(By.CLASS_NAME, 'fare').text,  # Extracts ticket price.
                "Seat_Availability": bus.find_element(By.XPATH, ".//div[contains(@class, 'seat-left')]").text if bus.find_elements(By.XPATH, ".//div[contains(@class, 'seat-left')]") else '0'
                 # Extracts seat availability or sets it to '0'.
            })
        except Exception as e:
            print(f"Error extracting bus details: {e}")
    return bus_details # Returns a list of dictionaries containing bus details.


# Scrape bus details from all pages
def scrape_all_pages(url, pages=3):
    driver = initialize_driver()  # Initializes the browser.
    all_details = [] # List to store all scraped bus details.

    try:
        for page in range(1, pages + 1): # Loops through pages (default: 3 pages).
            load_page(driver, url) # Loads the main page.
            if page > 1: # If it's not the first page:
                try:
                    # Find and click the pagination tab for the current page.
                    pagination = driver.find_element(By.XPATH, f"//div[contains(@class, 'DC_117_pageTabs') and text()='{page}']")
                    pagination.click()
                    time.sleep(5) # Waits for the next page to load.
                except Exception as e:
                    print(f"Error navigating to page {page}: {e}")
                    continue # Skips to the next iteration if there's an error.
            
            # Scrape routes and details for the current page
            for route_link, route_name in scrape_bus_routes(driver):
                all_details.extend(scrape_bus_details(driver, route_link, route_name))
    finally:
        driver.quit() # Closes the browser at the end of the scraping process.
    return all_details # Returns the complete list of scraped details.


# Main execution
if __name__ == "__main__":
    URL = "https://www.redbus.in/online-booking/west-bengal-transport-corporation?utm_source=rtchometile"
    bus_details = scrape_all_pages(URL) # Scrapes all pages for bus details.

    # Save the scraped data to a CSV file
    if bus_details:
        pd.DataFrame(bus_details).to_csv('WBengal_bus_details.csv', index=False) # Saves data to CSV.
        print("Scraping completed and data saved to 'WBengal_bus_details.csv'")
    else:
        print("No data scraped.")


Error navigating to page 2: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//div[contains(@class, 'DC_117_pageTabs') and text()='2']"}
  (Session info: chrome=131.0.6778.69); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00007FF73D342775+28773]
	(No symbol) [0x00007FF73D2AAFB0]
	(No symbol) [0x00007FF73D14552A]
	(No symbol) [0x00007FF73D198EAE]
	(No symbol) [0x00007FF73D19919C]
	(No symbol) [0x00007FF73D1E23F7]
	(No symbol) [0x00007FF73D1BEFDF]
	(No symbol) [0x00007FF73D1DF1A4]
	(No symbol) [0x00007FF73D1BED43]
	(No symbol) [0x00007FF73D18A548]
	(No symbol) [0x00007FF73D18B6B1]
	GetHandleVerifier [0x00007FF73D66F45D+3358029]
	GetHandleVerifier [0x00007FF73D68430D+3443709]
	GetHandleVerifier [0x00007FF73D6783FD+3394797]
	GetHandleVerifier [0x00007FF73D40929B+842635]
	(No symbol) [0x00007FF73D2B654F]
	(No symbol) [0x00007F

11. Bihar

In [ ]:
#Import Statements
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd

# Initialize the WebDriver
def initialize_driver():
    driver = webdriver.Chrome() # Opens the Chrome browser.
    driver.maximize_window() # Maximizes the browser window.
    return driver # Returns the driver instance.


# Load a webpage
def load_page(driver, url):
    driver.get(url) # Opens the specified URL in the browser.
    time.sleep(5)  # Waits for 5 seconds to ensure the page loads fully.


# Scrape all bus routes on the current page (or) Scrape Bus Routes
def scrape_bus_routes(driver):
    routes = driver.find_elements(By.CLASS_NAME, 'route')  # Finds all elements with the class `route`.
    route_links = [route.get_attribute('href') for route in routes] # Extracts href links from the elements.
    route_names = [route.text.strip() for route in routes] # Extracts the visible text (route names).
    return zip(route_links, route_names) # Returns a zipped object of links and names.


# Scrape bus details for a specific route
def scrape_bus_details(driver, url, route_name):
    driver.get(url) # Navigates to the route's URL.
    time.sleep(5) # Waits for the page to load.
    
    try:
        # Click "View Buses" button if available
        view_buses = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.CLASS_NAME, "button"))
        )
        view_buses.click()
        time.sleep(5) # Waits for buses to load.
        
        # Scroll to load buses  # Scrolls to the bottom of the page to load additional content.
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(5)
    except:
        print(f"No 'View Buses' button found for {route_name}")
        return [] # Returns an empty list if the button isn't found.
        

    # Collect bus details
    buses = driver.find_elements(By.CLASS_NAME, 'bus-item')  # Finds all bus items on the page.
    bus_details = []
    for bus in buses:  # Iterates through each bus element.
        try:
            bus_details.append({
                "Route_Name": route_name,
                "Route_Link": url,
                "Bus_Name": bus.find_element(By.CLASS_NAME, 'travels').text,  # Extracts bus name.
                "Bus_Type": bus.find_element(By.CLASS_NAME, 'bus-type').text,  # Extracts bus type.
                "Departing_Time": bus.find_element(By.CLASS_NAME, 'dp-time').text,  # Extracts departure time.
                "Duration": bus.find_element(By.CLASS_NAME, 'dur').text,  # Extracts travel duration.
                "Reaching_Time": bus.find_element(By.CLASS_NAME, 'bp-time').text, # Extracts arrival time.
                "Star_Rating": bus.find_element(By.CLASS_NAME, 'rating-sec').text if bus.find_elements(By.CLASS_NAME, 'rating-sec') else '0', # Extracts rating or sets it to '0'.
                "Price": bus.find_element(By.CLASS_NAME, 'fare').text,  # Extracts ticket price.
                "Seat_Availability": bus.find_element(By.XPATH, ".//div[contains(@class, 'seat-left')]").text if bus.find_elements(By.XPATH, ".//div[contains(@class, 'seat-left')]") else '0'
                 # Extracts seat availability or sets it to '0'.
            })
        except Exception as e:
            print(f"Error extracting bus details: {e}")
    return bus_details # Returns a list of dictionaries containing bus details.


# Scrape bus details from all pages
def scrape_all_pages(url, pages=3):
    driver = initialize_driver()  # Initializes the browser.
    all_details = [] # List to store all scraped bus details.

    try:
        for page in range(1, pages + 1): # Loops through pages (default: 3 pages).
            load_page(driver, url) # Loads the main page.
            if page > 1: # If it's not the first page:
                try:
                    # Find and click the pagination tab for the current page.
                    pagination = driver.find_element(By.XPATH, f"//div[contains(@class, 'DC_117_pageTabs') and text()='{page}']")
                    pagination.click()
                    time.sleep(5) # Waits for the next page to load.
                except Exception as e:
                    print(f"Error navigating to page {page}: {e}")
                    continue # Skips to the next iteration if there's an error.
            
            # Scrape routes and details for the current page
            for route_link, route_name in scrape_bus_routes(driver):
                all_details.extend(scrape_bus_details(driver, route_link, route_name))
    finally:
        driver.quit() # Closes the browser at the end of the scraping process.
    return all_details # Returns the complete list of scraped details.


# Main execution
if __name__ == "__main__":
    URL = "https://www.redbus.in/online-booking/bihar-state-road-transport-corporation-bsrtc/?utm_source=rtchometile"
    bus_details = scrape_all_pages(URL) # Scrapes all pages for bus details.

    # Save the scraped data to a CSV file
    if bus_details:
        pd.DataFrame(bus_details).to_csv('Bihar_bus_details.csv', index=False) # Saves data to CSV.
        print("Scraping completed and data saved to 'Bihar_bus_details.csv'")
    else:
        print("No data scraped.")
